In [12]:
#from google.colab import drive
#drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
cp: cannot stat '/content/drive/MyDrive/ECML2024-STA/data_test_output_and_data_1940_2023': No such file or directory


In [15]:
#!cp -r /content/drive/MyDrive/ECML2024-STA/data_test_output_and_data_1940_2023.zip .
#!unzip data_test_output_and_data_1940_2023.zip


Archive:  data_test_output_and_data_1940_2023.zip
  inflating: data_1940_2023/mslp_1940_2023.csv  
  inflating: data_1940_2023/README.md  
  inflating: data_1940_2023/sst_1940_2023.csv  
  inflating: data_1940_2023/ssta_1940_2023.csv  
  inflating: data_1940_2023/t2m_1940_2023.csv  
  inflating: data_test_output/README.md  
  inflating: data_test_output/test_output_phase_one.csv  
  inflating: data_test_output/test_output_phase_two.csv  


In [ ]:
#import torch
#
#!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
#!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
#!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
#!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
#!pip install git+https://github.com/pyg-team/pytorch_geometric.git
#!pip install graph-weather

In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 21.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
#from kan import *
from sklearn.model_selection import StratifiedKFold,KFold
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
import xgboost
import lightgbm
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model

In [68]:
locations = num_nodes = 5774
H = 9 #forecasting horizon

In [69]:
coords = pd.read_csv('coords.csv',header=None)

full_mlsp = pd.read_csv('data_1940_2023/mslp_1940_2023.csv',header=None).to_numpy()
full_sst = pd.read_csv('data_1940_2023/sst_1940_2023.csv',header=None).to_numpy()
full_ssta = pd.read_csv('data_1940_2023/ssta_1940_2023.csv',header=None).to_numpy()
full_t2m = pd.read_csv('data_1940_2023/t2m_1940_2023.csv',header=None).to_numpy()

train_mlsp = pd.read_csv('data_1940_2023/mslp_1940_2023.csv',header=None).to_numpy()
train_sst = pd.read_csv('data_1940_2023/sst_1940_2023.csv',header=None).to_numpy()
train_ssta = pd.read_csv('data_1940_2023/ssta_1940_2023.csv',header=None).to_numpy()
train_t2m = pd.read_csv('data_1940_2023/t2m_1940_2023.csv',header=None).to_numpy()


In [70]:
train_out = train_ssta[H:,:]
train_ssta = train_ssta[:-H,:]
train_t2m = train_t2m[:-H,:]
train_sst = train_sst[:-H,:]
train_mlsp = train_mlsp[:-H,:]

In [126]:
train_month = np.zeros((train_ssta.shape[0],1))
time = np.zeros((train_ssta.shape[0],1))
for i in range(train_ssta.shape[0]):
  train_month[i,0] = int((i+H)%12)
  time[i,0] = i/1008

In [72]:
coords
for i in range(locations):
  if coords.at[0,i] == 56 and coords.at[1,i] == 19.6875:
    print(1, i)
  if coords.at[0,i] == 58 and coords.at[1,i] == 19.6875:
    print(2, i)
  if coords.at[0,i] == 62 and coords.at[1,i] == 19.6875:
    print(3, i)
baltic_locations = [5640, 5690, 5773]

1 5640
2 5690
3 5773


In [73]:
train_t2m.shape

(999, 5774)

In [98]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import LinearRegression
def get_conf_int(alpha, lr, X, y):

    """
    Returns (1-alpha) 2-sided confidence intervals
    for sklearn.LinearRegression coefficients
    as a pandas DataFrame
    """

    coefs = lr.coef_
    X_aux = X.copy()
    dof = -np.diff(X_aux.shape)[0]
    mse = np.sum((y - lr.predict(X)) ** 2) / dof
    var_params = np.diag(np.linalg.inv(X_aux.T.dot(X_aux)))
    t_val = stats.t.isf(alpha/2, dof)
    gap = t_val * np.sqrt(mse * var_params)

    return {
        'lower': coefs - gap, 'upper': coefs + gap
    }

In [135]:
test_predictions = np.zeros((71, 5774))
for l in baltic_locations:
  print(l)
  index = l
  train_len = 1008-H-1*12
  val_len = 1008-H-train_len
  input = np.zeros((train_len,6))
  output = np.zeros((train_len,1))
  val_input = np.zeros((val_len,6))
  val_output = np.zeros((val_len,1))

  input[:,0] = train_mlsp[:train_len,index]
  input[:,1] = train_sst[:train_len,index]
  input[:,2] = train_t2m[:train_len,index]
  input[:,3] = train_ssta[:train_len,index]
  input[:,4] = train_month[:train_len,0]
  input[:,5] = time[:train_len,0]
  output[:,0] = train_out[:train_len,index]

  val_input[:,0] = train_mlsp[train_len:,index]
  val_input[:,1] = train_sst[train_len:,index]
  val_input[:,2] = train_t2m[train_len:,index]
  val_input[:,3] = train_ssta[train_len:,index]
  val_input[:,4] = train_month[train_len:,0]
  val_input[:,5] = time[train_len:,0]
  val_output[:,0] = train_out[train_len:,index]

  test_input = np.zeros((1,6))
  test_input[0,0] = full_mlsp[-1,index]
  test_input[0,1] = full_sst[-1,index]
  test_input[0,2] = full_t2m[-1,index]
  test_input[0,3] = full_ssta[-1,index]
  test_input[0,4] = int((1007+H)%12)
  test_input[0,5] = 1
  test_output = np.zeros((1,4))
  test_output_final = np.zeros((1,1))

  cross_val_output = np.zeros((val_len,4))
  test_val_output = np.zeros((val_len,1))
  cb_model = CatBoostRegressor(random_seed=42,verbose=False,use_best_model=True,iterations=100)
  cb_model.fit(input,output,eval_set=(val_input,val_output))
  cross_val_output[:,0] = cb_model.predict(val_input)
  test_output[:,0] = cb_model.predict(test_input)
  rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
  rf.fit(input,output)
  cross_val_output[:,1] = rf.predict(val_input)
  test_output[:,1] = rf.predict(test_input)
  lgb_model = lightgbm.LGBMRegressor(n_estimators = 100, random_state = 42,verbose= -1)
  lgb_model.fit(input,output,eval_set=(val_input,val_output))
  cross_val_output[:,2] = lgb_model.predict(val_input)
  test_output[:,2] = lgb_model.predict(test_input)
  xgb_model = xgboost.XGBRegressor(n_estimators=100,random_state=42)
  xgb_model.fit(input,output,eval_set=[(val_input,val_output)], verbose=False)#
  cross_val_output[:,3] = xgb_model.predict(val_input)
  test_output[:,3] = xgb_model.predict(test_input)

  meta_lasso = linear_model.LinearRegression(fit_intercept=False)#.Lasso(alpha=0.01,fit_intercept=False)#
  meta_lasso.fit(cross_val_output,val_output)
  CI = get_conf_int(0.5, meta_lasso, cross_val_output, val_output)
  test_val_output[:,0] = meta_lasso.predict(cross_val_output).flatten()
  test_output_final[:,0] = meta_lasso.predict(test_output)
  lower = np.dot(CI['lower'],test_output.T)
  upper = np.dot(CI['upper'],test_output.T)

  model_error1 = mean_squared_error(val_output, cross_val_output[:,0])
  model_error2 = mean_squared_error(val_output, cross_val_output[:,1])
  model_error3 = mean_squared_error(val_output, cross_val_output[:,2])
  model_error4 = mean_squared_error(val_output, cross_val_output[:,3])
  model_error = mean_squared_error(val_output, test_val_output[:,0])
  no_model_error = mean_squared_error(val_output, val_input[:,3])
  print(np.sqrt(model_error),np.sqrt(no_model_error),np.sqrt(model_error1),np.sqrt(model_error2),np.sqrt(model_error3),np.sqrt(model_error4))
  test_predictions[:,index] = test_output_final[:,0]

  print(test_output_final,lower,upper)



5640


<ipython-input-135-17ec34a574d6>:45: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(input,output)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:300: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


0.3335793880819365 1.02546097366614 0.6711716869988532 0.7422762449339873 0.8934211808132705 0.7932030829627355
[[1.4317156]] [[-0.79328741]] [[3.65671861]]
5690


<ipython-input-135-17ec34a574d6>:45: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(input,output)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:300: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


0.5624051909849404 1.423937092063874 0.8781282771105161 0.9906351913830184 0.944345820515879 0.8522864537215075
[[2.59397936]] [[-1.52689996]] [[6.71485868]]
5773


<ipython-input-135-17ec34a574d6>:45: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(input,output)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:300: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


0.5966411353856652 1.3373376429106576 0.8204044595702467 0.7987016816343239 0.9037005631088818 0.6805835342669966
[[1.56377272]] [[-1.44704371]] [[4.57458915]]
